### In this file we will go through the Devitt el al. paper (arXiv: 0905.2794v4) and derive and explain all of the math and theory to fully understand basic QEC

## ll. Preliminaries

## lll. Quantum Errors: Cause and Effect

## IV. The 3-Qubit Code: A good starting point for quantum error correction

## V. The 9-Qubit Code: The first full quantum code

## Vl. Quantum Error Detection

## Vll. Stabiliser Formalism

## Vlll. Quantum Error Correction with Stabiliser Codes

## lX. Digitization of Quantum Errors

## X. Fault Tolerant Quantum Error Correction and the Threashold Theorem

## Xl. Fault Tolerant Operations on Encoded Data

## Xll. Fault Tolerant Gate Design for Logical State Preparation

## Xlll. Loss Protection

## XlV. Some Modern Develpments in Quantum Error Correction